In [29]:
#제출용

#필요한 모듈 불러오기~
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
import pymysql
# conn=pymysql.connect(host='127.0.0.1',user='root',password='mysql904#',db='mydb',charset='utf8')

# cur=conn.cursor()

# cur.execute("create table navermap(num int,name varchar(50) ,major varchar(50) ,address varchar(50),tel varchar(50),open varchar(50),opentime varchar(50),visitrev varchar(50),blogrev varchar(50),star varchar(50),link varchar(100))")
serv = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=serv)

# 네이버 지도 페이지 열기
driver.get(f"https://map.naver.com/p?c=15.00,0,0,3,dh")
time.sleep(1)

# 검색어 입력
keyword = input('검색어를 입력해주세요: \n')
search = driver.find_element(By.CSS_SELECTOR, '.input_search')
search.send_keys(f'{keyword}')
search.send_keys(Keys.ENTER)
time.sleep(2)

# html 안의 html로 들어가기
driver.switch_to.frame("searchIframe") 
time.sleep(2)

# 클릭할 가게 이름 찾아오기
keyword = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")

j = 1 #카운터용
DATA =[]
print(f"\n총 {len(keyword)}개를 검색하였습니다\n")
for i in range(len(keyword)): #매장개수 만큼 클릭하게 설정
    keyword[i].find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").click()  #매장 클릭
    time.sleep(2)
    
    driver.switch_to.default_content()  # 먼저 기존의 iframe에서 빠져나오기
    driver.switch_to.frame("entryIframe") #새로운 html 들어가기
    time.sleep(2)
   # navermap 테이블 , 필드명 ( name,major,address,tel,open,opentime,visitrev,blogrev,star) 
   # 크롤링 해오기
    이름 = driver.find_element(By.CSS_SELECTOR, ".GHAhO").text.strip()
    업종 = driver.find_element(By.CSS_SELECTOR, ".lnJFt").text.strip()
    주소 = driver.find_element(By.CSS_SELECTOR, ".LDgIH").text.strip()
    번호 = driver.find_element(By.CSS_SELECTOR, ".xlx7Q").text.strip()
    영업 = driver.find_element(By.CSS_SELECTOR, ".y6tNq em").text.strip()
    시간 = driver.find_element(By.CSS_SELECTOR, ".y6tNq time").text.strip()
    방문 = driver.find_element(By.CSS_SELECTOR, ".PXMot a[href*='/review/visitor']").text.strip()
    리뷰 = driver.find_element(By.CSS_SELECTOR, ".PXMot a[href*='/review/ugc']").text.strip()
    try: # 별점이 없는 경우 
        별점 = driver.find_element(By.CSS_SELECTOR, ".PXMot.LXIwF").text.strip().replace("\n", "").replace("별점", "★")
    except: # 링크 없는 경우 처리하기 위함
        별점 = "별점이 없습니다"
    try: # 링크가 없는 경우가 있음 
        링크 = driver.find_element(By.CSS_SELECTOR, ".place_bluelink.CHmqa").get_attribute("href")
    except: # 링크 없는 경우 처리하기 위함
        링크 = "URL이 없습니다"    
    time.sleep(2)
    print(f"{j}. 매장명: {이름}\n업종: {업종}\n주소: {주소}\n연락처: {번호}\n영업현황: {영업}\n영업시간: {시간}\n별점: {별점} / {방문} / {리뷰}\n링크: {링크}")
    print("="*40)
    cur.execute("insert into navermap values('%d','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s')"%(j,이름,업종,주소,번호,영업,시간,방문,리뷰,별점,링크))
    DATA.append([이름,업종,주소,번호,영업,시간,별점,방문,리뷰,링크])
    j += 1    
    driver.switch_to.default_content()  # 기존 html 나가기
    driver.switch_to.frame("searchIframe") 
    time.sleep(2)
conn.commit()
df = pd.DataFrame(DATA, columns= ['매장명','업종','주소','연락처','영업현황','영업시간','별점','방문자 리뷰','블로그 리뷰','링크']) 
df.set_index("매장명",inplace=True)
df.to_csv("네이버맛집1.csv",encoding="utf-8")
df.to_csv("네이버맛집2.csv",encoding="utf-8-sig") 
df.to_csv("네이버맛집3.csv",encoding="euc-kr") 

# 드라이버 종료 끝 ㅅㄱ
driver.quit()

검색어를 입력해주세요: 
 이수역 맛집



총 10개를 검색하였습니다

1. 매장명: 정선콩국수 소소정
업종: 한식
주소: 서울 서초구 동광로 68 1층 정선콩국수 소소정
연락처: 0507-1433-8667
영업현황: 영업 중
영업시간: 15:00에 브레이크타임
별점: 별점이 없습니다 / 방문자 리뷰 616 / 블로그 리뷰 207
링크: https://blog.naver.com/jin508815
2. 매장명: 지즐미육회관 사당점
업종: 소고기구이
주소: 서울 서초구 방배천로2길 39 1층
연락처: 0507-1492-3356
영업현황: 영업 중
영업시간: 21:00에 라스트오더
별점: ★4.73 / 방문자 리뷰 618 / 블로그 리뷰 152
링크: https://blog.naver.com/hanafc3334
3. 매장명: 애플하우스
업종: 떡볶이
주소: 서울 동작구 동작대로27다길 29 2층
연락처: 02-595-1629
영업현황: 영업 중
영업시간: 20:30에 라스트오더
별점: ★4.34 / 방문자 리뷰 12,513 / 블로그 리뷰 8,571
링크: URL이 없습니다
4. 매장명: 외양간
업종: 육류,고기요리
주소: 서울 동작구 동작대로11길 16 1층
연락처: 0507-1484-8223
영업현황: 영업 전
영업시간: 15:00에 영업 시작
별점: ★4.69 / 방문자 리뷰 595 / 블로그 리뷰 702
링크: https://www.instagram.com/cowhome_sadang/
5. 매장명: 팔각도 이수역점
업종: 닭갈비
주소: 서울 동작구 동작대로27다길 12 1층
연락처: 0507-1340-0420
영업현황: 영업 중
영업시간: 22:10에 라스트오더
별점: ★4.72 / 방문자 리뷰 4,696 / 블로그 리뷰 1,312
링크: https://8gakdo.co.kr/
6. 매장명: 목구멍 이수점
업종: 육류,고기요리
주소: 서울 동작구 동작대로27길 16-4
연락처: 0507-1417-1959
영업현황: 영업 전
영업시간: 16:00에 영업 시작
별점: 별점이 없습니다 / 방문자 리뷰 2

In [19]:
conn.close()